In [1]:
import numpy as np
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import JointState
from ikpy.chain import Chain
from scipy.spatial.transform import Rotation as R
from geometry_msgs.msg import PoseStamped, TransformStamped
import tf2_ros
import time
from pymycobot.mycobot import MyCobot
import math

In [3]:
# 원하는 Pose와 회전행렬 정의
pos = [0.31127507, -0.11197324,  0.36560894]

rot_mat = np.array([
    [-0.015,  0.375, -0.927],
    [ 0.998,  0.064,  0.010],
    [ 0.063, -0.925, -0.375]
])

# tcp를 태그까지 보내기 위해 오프셋 설정
tcp_offset = np.array([0.09, 0.0, 0.0])  # z축으로 11cm 앞으로 이동
print("초기 pos:", pos)
pos = np.array(pos)

print(f"계산한 tcp 오프셋: {rot_mat @ tcp_offset}")
pos -= rot_mat @ tcp_offset  # 회전행렬을 이용해 오프셋 적용
print("오프셋 적용된 pos:", pos)

초기 pos: [0.31127507, -0.11197324, 0.36560894]
계산한 tcp 오프셋: [-0.00135  0.08982  0.00567]
오프셋 적용된 pos: [ 0.31262507 -0.20179324  0.35993894]


In [3]:
# 조인트 이름 정의
joint_names = [
            "link2_to_link1", "link3_to_link2", "link4_to_link3",
            "link5_to_link4", "link6_to_link5", "link6output_to_link6"
        ]

In [4]:
# URDF 경로 및 체인 생성
URDF_PATH = "/home/jetcobot/silver_ws/src/jetcobot_movetag/urdf/mycobot_280_m5/mycobot_280m5_with_gripper_parallel.urdf"
chain = Chain.from_urdf_file(URDF_PATH, base_elements=["g_base"])

/home/jetcobot/.local/lib/python3.12/site-packages/ikpy/urdf/URDF.py:261: UserWarning: Joint g_base_to_link1 is of type: fixed, but has an 'axis' attribute defined. This is not in the URDF spec and thus this axis is ignored
  warnings.warn("Joint {} is of type: fixed, but has an 'axis' attribute defined. This is not in the URDF spec and thus this axis is ignored".format(joint.attrib["name"]))
/home/jetcobot/.local/lib/python3.12/site-packages/ikpy/chain.py:60: UserWarning: Link Base link (index: 0) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive".format(link.name, link_index))
/home/jetcobot/.local/lib/python3.12/site-packages/ikpy/chain.py:60: UserWarning: Link g_base_to_link1 (index: 1) is of t

In [5]:
chain

Kinematic chain name=chain links=['Base link', 'g_base_to_link1', 'link2_to_link1', 'link3_to_link2', 'link4_to_link3', 'link5_to_link4', 'link6_to_link5', 'link6output_to_link6', 'link6output_to_gripper_base', 'gripper_base_to_jetcocam'] active_links=[ True  True  True  True  True  True  True  True  True  True]

In [6]:
# 조인트 인덱스 찾기
joint_indices = [idx for idx, link in enumerate(chain.links) if link.name in joint_names]
print("joint_indices:", joint_indices)
print("URDF Base Link Name:", chain.links[0].name)
print("ikpy Chain:", chain)

joint_indices: [2, 3, 4, 5, 6, 7]
URDF Base Link Name: Base link
ikpy Chain: Kinematic chain name=chain links=['Base link', 'g_base_to_link1', 'link2_to_link1', 'link3_to_link2', 'link4_to_link3', 'link5_to_link4', 'link6_to_link5', 'link6output_to_link6', 'link6output_to_gripper_base', 'gripper_base_to_jetcocam'] active_links=[ True  True  True  True  True  True  True  True  True  True]


In [6]:
# IK 계산
ik_result = chain.inverse_kinematics(pos, target_orientation=rot_mat)
print("ikpy 결과(전체):", ik_result)
radian_list = [ik_result[idx] for idx in joint_indices]
print("각 조인트 각도(radian):", radian_list)

ikpy 결과(전체): [ 0.00000000e+00 -5.92928334e-01  2.26468967e-01 -9.40708247e-01
  2.27542418e-05  5.06498123e-01 -5.72841173e-01 -7.85151475e-01
  0.00000000e+00  0.00000000e+00]
각 조인트 각도(radian): [0.22646896699690527, -0.9407082474659828, 2.2754241813580904e-05, 0.5064981234039198, -0.5728411727563697, -0.7851514753737463]
